In [1]:
import math
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.distributions
from collections import namedtuple
from itertools import count

device = "cuda:0"
floattype = torch.float

batchsize = 512
nsamples = 8
npoints = 5
emsize = 512


class Graph_Transformer(nn.Module):
    def __init__(self, emsize = 64, nhead = 8, nhid = 1024, nlayers = 4, ndecoderlayers = 2, dropout = 0):
        super().__init__()
        self.emsize = emsize
        from torch.nn import TransformerEncoder, TransformerEncoderLayer, TransformerDecoder, TransformerDecoderLayer
        encoder_layers = TransformerEncoderLayer(emsize, nhead, nhid, dropout = dropout)
        decoder_layers = TransformerDecoderLayer(emsize, nhead, nhid, dropout = dropout)
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)
        self.transformer_decoder = TransformerDecoder(decoder_layers, ndecoderlayers)
        self.encoder = nn.Linear(2, emsize)
        self.outputattention_query = nn.Linear(emsize, emsize, bias = False)
        self.outputattention_key = nn.Linear(emsize, emsize, bias = False)
        self.start_token = nn.Parameter(torch.randn([emsize], device = device))
    
    def generate_subsequent_mask(self, sz): #last dimension will be softmaxed over when adding to attention logits, if boolean the ones turn into -inf
        mask = (torch.triu(torch.ones(sz, sz, device = device)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        
        #mask = torch.triu(torch.ones([sz, sz], dtype = torch.bool, device = device), diagonal = 1)
        return mask
    
    def encode(self, src): #src must be [batchsize * nsamples, npoints, 3]
        src = self.encoder(src).transpose(0, 1)
        output = self.transformer_encoder(src)
        return output #[npoints, batchsize * nsamples, emsize]
    
    def decode_next(self, memory, tgt, route_mask): #route mask is [batchsize * nsamples, npoints], both memory and tgt must have batchsize and nsamples in same dimension (the 1th one)
        npoints = memory.size(0)
        batchsize = tgt.size(1)
        """if I really wanted this to be efficient I'd only recompute the decoder for the last tgt, and just remebering what the others looked like from before (won't change due to mask)"""
        """have the option to freeze the autograd on all but the last part of tgt, although at the moment this is a very natural way to say: initial choices matter more"""
        tgt_mask = self.generate_subsequent_mask(tgt.size(0))
        output = self.transformer_decoder(tgt, memory, tgt_mask) #[tgt, batchsize * nsamples, emsize]
        output_query = self.outputattention_query(memory).transpose(0, 1) #[batchsize * nsamples, npoints, emsize]
        output_key = self.outputattention_key(output[-1]) #[batchsize * nsamples, emsize]
        output_attention = torch.matmul(output_query * self.emsize ** -0.5, output_key.unsqueeze(-1)).squeeze(-1) #[batchsize * nsamples, npoints], technically don't need to scale attention as we divide by variance next anyway
        output_attention_tanh = output_attention.tanh() #[batchsize * nsamples, npoints]
        
        #we clone the route_mask incase we want to backprop using it (else it was modified by inplace opporations)
        output_attention = output_attention.masked_fill(route_mask.clone(), float('-inf')) #[batchsize * nsamples, npoints]
        output_attention_tanh = output_attention_tanh.masked_fill(route_mask.clone(), float('-inf')) #[batchsize * nsamples, npoints]
        
        return output_attention_tanh, output_attention #[batchsize * nsamples, npoints]
    
    def calculate_logprob(self, memory, routes): #memory is [npoints, batchsize * nsamples, emsize], routes is [batchsize * nsamples, npoints - 4], rather than backproping the entire loop, this saves vram (and computation)
        npoints = memory.size(0)
        ninternalpoints = routes.size(1)
        bigbatchsize = memory.size(1)
        memory_ = memory.gather(0, routes.transpose(0, 1).unsqueeze(2).expand(-1, -1, self.emsize)) #[npoints - 4, batchsize * nsamples, emsize] reorder memory into order of routes
        tgt = torch.cat([self.start_token.unsqueeze(0).unsqueeze(1).expand(1, bigbatchsize, -1), memory_[:-1]]) #[npoints - 4, batchsize * nroutes, emsize], want to go from memory to tgt
        tgt_mask = self.generate_subsequent_mask(ninternalpoints)
        output = self.transformer_decoder(tgt, memory, tgt_mask) #[npoints - 4, batchsize * nsamples, emsize]
        """want probability of going from key to query, but first need to normalise (softmax with mask)"""
        output_query = self.outputattention_query(memory_).transpose(0, 1) #[batchsize * nsamples, npoints - 4, emsize]
        output_key = self.outputattention_key(output).transpose(0, 1) #[batchsize * nsamples, npoints - 4, emsize]
        attention_mask = torch.full([ninternalpoints, ninternalpoints], True, device = device).triu(1) #[npoints - 4, npoints - 4], True for i < j
        output_attention = torch.matmul(output_query * self.emsize ** -0.5, output_key.transpose(-1, -2))
        """quick fix to stop divergence"""
        output_attention_tanh = output_attention.tanh()
        
        output_attention_tanh = output_attention_tanh.masked_fill(attention_mask, float('-inf'))
        output_attention_tanh = output_attention_tanh - output_attention_tanh.logsumexp(-2, keepdim = True) #[batchsize * nsamples, npoints - 4, npoints - 4]
        
        output_attention = output_attention.masked_fill(attention_mask, float('-inf'))
        output_attention = output_attention - output_attention.logsumexp(-2, keepdim = True) #[batchsize * nsamples, npoints - 4, npoints - 4]
        
        """infact I'm almost tempted to not mask choosing a previous point, so it's forced to learn it and somehow incorporate it into its computation, but without much impact on reinforcing good examples"""
        logprob_tanh = output_attention_tanh.diagonal(dim1 = -1, dim2 = -2).sum(-1) #[batchsize * nsamples]
        logprob = output_attention.diagonal(dim1 = -1, dim2 = -2).sum(-1) #[batchsize * nsamples]
        return logprob_tanh, logprob #[batchsize * nsamples]

NN = Graph_Transformer().to(device)
optimizer = optim.Adam(NN.parameters())


class environment:    
    def reset(self, npoints, batchsize, nsamples=1):
        self.batchsize = (
            batchsize * nsamples
        )  # so that I don't have to rewrite all this code, we store these two dimensions together
        self.nsamples = nsamples
        self.npoints = npoints
        self.points = (
            torch.rand([batchsize, npoints, 2], dtype = floattype, device=device)
            .unsqueeze(1)
            .expand(-1, nsamples, -1, -1)
            .reshape(self.batchsize, npoints, 2)
        )
        
        self.distance_matrix = (self.points.unsqueeze(1) - self.points.unsqueeze(2)).square().sum(-1).sqrt() # [batchsize * nsamples, npoints, npoints]
        
        self.previous_point = None
        
        self.points_mask = torch.zeros(
                    [self.batchsize, npoints], dtype=torch.bool, device=device
                )
        self.points_sequence = torch.empty(
            [self.batchsize, 0], dtype=torch.long, device=device
        )
        
        self.cost = torch.zeros([self.batchsize], dtype = floattype, device=device)

        self.logprob = torch.zeros([self.batchsize], dtype = floattype, device=device, requires_grad=True)

    def update(self, point_index):  # point_index is [batchsize]
        
        assert list(point_index.size()) == [self.batchsize]
        assert str(point_index.device) == device
        assert self.points_mask.gather(1, point_index.unsqueeze(1)).sum() == 0
        
        if self.previous_point != None:
            self.cost += self.distance_matrix.gather(2, self.previous_point.unsqueeze(1).unsqueeze(2).expand(-1, self.npoints, 1)).squeeze(2).gather(1, point_index.unsqueeze(1)).squeeze(1)
        
        self.previous_point = point_index
        self.points_mask.scatter_(1, point_index.unsqueeze(1), True)
        self.points_sequence = torch.cat([self.points_sequence, point_index.unsqueeze(1)], dim = 1)
        
        return
    
    def sample_point(self, logits): #logits must be [batchsize * nsamples, npoints]
        probs = torch.distributions.categorical.Categorical(logits = logits)
        next_point = probs.sample() #size is [batchsize * nsamples]
        self.update(next_point)
        self.logprob = self.logprob + probs.log_prob(next_point)
        return next_point #[batchsize * nsamples]
    
    def sampleandgreedy_point(self, logits): #logits must be [batchsize * nsamples, npoints], last sample will be the greedy choice (but we still need to keep track of its logits)
        logits_sample = logits.view(-1, self.nsamples, self.npoints)[:, :-1, :]
        probs = torch.distributions.categorical.Categorical(logits = logits_sample)
        
        sample_point = probs.sample() #[batchsize, (nsamples - 1)]
        greedy_point = logits.view(-1, self.nsamples, self.npoints)[:, -1, :].max(-1, keepdim = True)[1] #[batchsize, 1]
        next_point = torch.cat([sample_point, greedy_point], dim = 1).view(-1)
        self.update(next_point)
        self.logprob = self.logprob + torch.cat([probs.log_prob(sample_point), torch.zeros([sample_point.size(0), 1], device = device)], dim = 1).view(-1)
        return next_point
    
    def laststep(self):
        
        assert self.points_sequence.size(1) == self.npoints
        
        self.cost += self.distance_matrix.gather(2, self.points_sequence[:, 0].unsqueeze(1).unsqueeze(2).expand(-1, self.npoints, 1)).squeeze(2).gather(1, self.points_sequence[:, -1].unsqueeze(1)).squeeze(1)
    

env = environment()


def train(epochs = 30000, npoints = 10, batchsize = 100, nsamples = 8, negative_cutoff = 1):
    NN.train()
    for i in range(epochs):
        env.reset(npoints, batchsize, nsamples)
        """include the boundary points, kinda makes sense that they should contribute (atm only in the encoder, difficult to see how in the decoder)"""
        memory = NN.encode(env.points) #[npoints, batchsize * nsamples, emsize]
        #### #### #### remember to include tgt.detach() when reinstate with torch.no_grad()
        tgt = NN.start_token.unsqueeze(0).unsqueeze(1).expand(1, batchsize * nsamples, -1).detach() #[1, batchsize * nsamples, emsize]
        #with torch.no_grad(): #to speed up computation, selecting routes is done without gradient
        with torch.no_grad():
            for j in range(0, npoints):
                #### #### #### remember to include memory.detach() when reinstate with torch.no_grad()
                _, logits = NN.decode_next(memory.detach(), tgt, env.points_mask)
                next_point = env.sampleandgreedy_point(logits)
                """
                for inputing the previous embedding into decoder
                """
                tgt = torch.cat([tgt, memory.gather(0, next_point.unsqueeze(0).unsqueeze(2).expand(1, -1, memory.size(2)))]) #[nsofar, batchsize * nsamples, emsize]
                """
                for inputing the previous decoder output into the decoder (allows for an evolving strategy, but doesn't allow for fast training
                """
                ############

        env.laststep()
        
        NN.eval()
        _, logprob = NN.calculate_logprob(memory, env.points_sequence) #[batchsize * nsamples]
        NN.train()
        """
        clip logprob so doesn't reinforce things it already knows
        TBH WANT SOMETHING DIFFERENT ... want to massively increase training if find something unexpected and otherwise not
        """
        greedy_prob = logprob.view(batchsize, nsamples)[:, -1].detach() #[batchsize]
        greedy_baseline = env.cost.view(batchsize, nsamples)[:, -1] #[batchsize], greedy sample
        fixed_baseline = 0.5 * torch.ones([1], device = device)
        min_baseline = env.cost.view(batchsize, nsamples)[:, :-1].min(-1)[0] #[batchsize], minimum cost
        baseline = greedy_baseline
        positive_reinforcement = - F.relu( - (env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1))) #don't scale positive reinforcement
        negative_reinforcement = F.relu(env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1))
        positive_reinforcement_binary = env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1) <= -0.05
        negative_reinforcement_binary = env.cost.view(batchsize, nsamples)[:, :-1] - baseline.unsqueeze(1) > negative_cutoff
        """
        binary positive reinforcement
        """
        #loss = - ((logprob.view(batchsize, nsamples)[:, :-1] < -0.2) * logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement_binary).mean() #+ (logprob.view(batchsize, nsamples)[:, :-1] > -1) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement_binary
        """
        clipped binary reinforcement
        """
        #loss = ( 
        #        - logprob.view(batchsize, nsamples)[:, :-1] 
        #        #* (logprob.view(batchsize, nsamples)[:, :-1] < 0) 
        #        * positive_reinforcement_binary 
        #        + logprob.view(batchsize, nsamples)[:, :-1] 
        #        * (logprob.view(batchsize, nsamples)[:, :-1] > greedy_prob.unsqueeze(1) - 80) 
        #        * negative_reinforcement_binary 
        #).mean()
        """
        clipped binary postive, clipped weighted negative
        """
        #loss = ( - logprob.view(batchsize, nsamples)[:, :-1] * (logprob.view(batchsize, nsamples)[:, :-1] < -0.2) * positive_reinforcement_binary + logprob.view(batchsize, nsamples)[:, :-1] * (logprob.view(batchsize, nsamples)[:, :-1] > -2) * negative_reinforcement ).mean()
        """
        clipped reinforcement without rescaling
        """
        #loss = ((logprob.view(batchsize, nsamples)[:, :-1] < -0.7) * logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement + (logprob.view(batchsize, nsamples)[:, :-1] > -5) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement).mean()
        """
        clipped reinforcement
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * positive_reinforcement / (positive_reinforcement.var() + 0.001).sqrt() + (logprob.view(batchsize, nsamples)[:, :-1] > -3) * logprob.view(batchsize, nsamples)[:, :-1] * negative_reinforcement / (negative_reinforcement.var() + 0.001).sqrt()).mean()
        """
        balanced reinforcement
        """
        #loss = (logprob.view(batchsize, nsamples)[:, :-1] * (positive_reinforcement / (positive_reinforcement.var() + 0.001).sqrt() + negative_reinforcement / (negative_reinforcement.var() + 0.001).sqrt())).mean()
        """
        regular loss
        """
        loss = (logprob.view(batchsize, nsamples)[:, :-1] * (positive_reinforcement + negative_reinforcement)).mean()
        optimizer.zero_grad()
        loss.backward()
        #print(NN.encoder.weight.grad)
        optimizer.step()
        #print(greedy_baseline.mean().item())
        print(greedy_baseline.mean().item(), logprob.view(batchsize, nsamples)[:, -1].mean().item(), logprob.view(batchsize, nsamples)[:, :-1].mean().item(), logprob[batchsize - 1].item(), logprob[0].item(), env.logprob[0].item())
        
   

In [2]:
train(300000, 50, 400, 8, 1)

19.33003044128418 -146.136474609375 -148.4093780517578 -146.40731811523438 -148.34957885742188 -148.3495635986328
14.266571044921875 -141.11985778808594 -147.7121124267578 -140.7115478515625 -147.5675048828125 -147.56748962402344
14.759800910949707 -120.56681060791016 -137.9326629638672 -121.71231079101562 -133.65328979492188 -133.65328979492188
13.19284439086914 -95.56456756591797 -117.49597930908203 -98.23407745361328 -118.01054382324219 -118.01053619384766
9.876015663146973 -101.44986724853516 -126.246337890625 -97.87459564208984 -132.3051300048828 -132.30514526367188
12.03374195098877 -83.46735382080078 -105.8130111694336 -87.72215270996094 -105.43649291992188 -105.43648529052734
12.128379821777344 -80.26680755615234 -103.36506652832031 -83.88217163085938 -106.64299774169922 -106.64299774169922
11.223166465759277 -76.7909927368164 -101.11714172363281 -75.93172454833984 -99.51309204101562 -99.5130615234375
9.960550308227539 -66.40434265136719 -91.32095336914062 -68.78376007080078 -9

7.349706649780273 -17.78327178955078 -31.465469360351562 -17.817176818847656 -26.24360466003418 -26.243610382080078
7.234799861907959 -17.48845672607422 -31.27753257751465 -22.975181579589844 -23.47623634338379 -23.47623062133789
7.0589799880981445 -16.931303024291992 -30.47750473022461 -16.198190689086914 -46.34967803955078 -46.34967803955078
6.989931583404541 -15.8538818359375 -28.983814239501953 -16.586029052734375 -31.43427848815918 -31.434276580810547
6.978456974029541 -15.694719314575195 -28.385976791381836 -19.445758819580078 -25.35430145263672 -25.354299545288086
6.999050617218018 -15.45067310333252 -28.465423583984375 -14.723217964172363 -28.52054214477539 -28.520530700683594
6.895200252532959 -15.701408386230469 -28.868518829345703 -15.743189811706543 -19.96455192565918 -19.964550018310547
6.861154079437256 -15.283720016479492 -28.291454315185547 -15.62879467010498 -27.030494689941406 -27.030467987060547
6.809391498565674 -14.937609672546387 -27.718908309936523 -14.1767978668

6.50493860244751 -8.65223217010498 -16.757326126098633 -6.61370325088501 -16.105363845825195 -16.105348587036133
6.435307502746582 -8.537440299987793 -16.412500381469727 -9.266490936279297 -26.11478614807129 -26.11479377746582
6.458516597747803 -8.46693229675293 -16.434785842895508 -9.517578125 -13.579084396362305 -13.579086303710938
6.486640930175781 -8.399648666381836 -16.228731155395508 -5.151573181152344 -15.31338882446289 -15.313384056091309
6.458113193511963 -8.285188674926758 -16.186893463134766 -8.554869651794434 -15.163000106811523 -15.162989616394043
6.489268779754639 -8.398886680603027 -16.318334579467773 -6.621795177459717 -16.829689025878906 -16.829710006713867
6.4669365882873535 -8.413708686828613 -16.378782272338867 -6.572201728820801 -12.979625701904297 -12.979629516601562
6.450483322143555 -8.373091697692871 -16.323598861694336 -6.680539131164551 -17.823410034179688 -17.823400497436523
6.478993892669678 -8.495205879211426 -16.44256591796875 -8.97665786743164 -20.077022

6.424017906188965 -6.701207160949707 -13.054659843444824 -5.761398792266846 -12.478910446166992 -12.478907585144043
6.4453606605529785 -6.509206295013428 -12.562358856201172 -5.167810440063477 -13.813067436218262 -13.813089370727539
6.4959001541137695 -6.482944965362549 -12.66742992401123 -6.447569847106934 -7.563160419464111 -7.5631608963012695
6.393218040466309 -6.580146312713623 -12.874239921569824 -6.856791973114014 -13.603631019592285 -13.603647232055664
6.42982292175293 -6.549321174621582 -12.837151527404785 -3.8388750553131104 -12.885007858276367 -12.885011672973633
6.466092586517334 -6.797470569610596 -13.2374849319458 -4.371448516845703 -8.721896171569824 -8.721891403198242
6.44587516784668 -6.597377300262451 -12.844226837158203 -4.847912788391113 -13.447333335876465 -13.447334289550781
6.430847644805908 -6.396340370178223 -12.515007019042969 -5.188965797424316 -9.52888298034668 -9.528868675231934
6.447946548461914 -6.337051868438721 -12.545817375183105 -5.7302093505859375 -11

6.42296838760376 -5.762937068939209 -11.171730041503906 -6.367124557495117 -7.822014331817627 -7.822031497955322
6.395906925201416 -5.774982929229736 -11.21397876739502 -4.4924774169921875 -7.223455429077148 -7.2234697341918945
6.3679680824279785 -5.636014461517334 -11.03567123413086 -8.73355770111084 -12.465396881103516 -12.465400695800781
6.379608154296875 -5.836000919342041 -11.344263076782227 -7.4495744705200195 -16.740419387817383 -16.740436553955078
6.363124847412109 -5.893789768218994 -11.63676929473877 -5.137575149536133 -10.743850708007812 -10.743842124938965
6.396153450012207 -5.845457553863525 -11.52864933013916 -7.913110733032227 -10.465043067932129 -10.465065002441406
6.371052265167236 -5.756358623504639 -11.439305305480957 -5.120235443115234 -11.633796691894531 -11.633796691894531
6.354645252227783 -5.7805304527282715 -11.301883697509766 -7.230766296386719 -16.568824768066406 -16.568811416625977
6.405754089355469 -5.70701265335083 -11.122907638549805 -3.7142016887664795 -

...

6.030551433563232 -2.3812191486358643 -4.624149799346924 -1.248270034790039 -7.5692596435546875 -7.5692901611328125
6.056416034698486 -2.2755300998687744 -4.551759243011475 -0.7900810241699219 -5.517099380493164 -5.517023086547852
6.027668476104736 -2.3018112182617188 -4.607569217681885 -2.133350372314453 -3.6969375610351562 -3.6969680786132812
6.059947490692139 -2.27689266204834 -4.523604393005371 -1.379974365234375 -6.331153869628906 -6.3311309814453125
6.016168117523193 -2.2416305541992188 -4.498420238494873 -4.661170959472656 -1.5457038879394531 -1.5456733703613281
6.021347522735596 -2.292048692703247 -4.5627121925354 -4.8507843017578125 -3.6234922409057617 -3.623538017272949
6.036349773406982 -2.2251455783843994 -4.383234977722168 -0.66265869140625 -3.4762535095214844 -3.4762611389160156
6.023169040679932 -2.2928578853607178 -4.535923004150391 -1.5513429641723633 -5.995487213134766 -5.9954833984375
6.054195404052734 -2.3561599254608154 -4.639978885650635 -1.4018936157226562 -3.828

6.087625980377197 -2.193477153778076 -4.407115936279297 -2.2352752685546875 -1.6598917245864868 -1.6598840951919556
6.059452533721924 -2.349642276763916 -4.623088359832764 -3.7745704650878906 -3.95050048828125 -3.95050048828125
6.064529895782471 -2.310576915740967 -4.57757043838501 -3.007068634033203 -10.645018577575684 -10.645049095153809
6.072282791137695 -2.374436855316162 -4.6751179695129395 -3.437347412109375 -6.092987060546875 -6.092994689941406
6.028260231018066 -2.2571280002593994 -4.4633259773254395 -3.9361953735351562 -4.60394287109375 -4.60394287109375
6.0176591873168945 -2.1545705795288086 -4.186982154846191 -3.4924774169921875 -6.002010345458984 -6.002040863037109
6.073052883148193 -1.996539831161499 -4.0293145179748535 -1.0962295532226562 -2.245208740234375 -2.245269775390625
6.082691669464111 -2.0503463745117188 -3.985731601715088 -3.314685821533203 -9.398246765136719 -9.3983154296875
6.09188175201416 -2.0297558307647705 -4.107326030731201 -3.07940673828125 -6.6874265670

6.035006999969482 -2.222302198410034 -4.320028781890869 -2.7564544677734375 -1.3267822265625 -1.3267669677734375
6.027291297912598 -2.1183176040649414 -4.250256061553955 -0.407257080078125 -3.4328155517578125 -3.4328041076660156
6.052084922790527 -2.1418426036834717 -4.279172897338867 -2.5140380859375 -1.8045425415039062 -1.8045730590820312
6.014750957489014 -2.162362813949585 -4.330291748046875 -3.4791312217712402 -1.5287219285964966 -1.5286990404129028
6.010194778442383 -2.2634592056274414 -4.47779655456543 -2.3000259399414062 -5.013877868652344 -5.013896942138672
6.042282581329346 -2.231218099594116 -4.354273319244385 -2.9033126831054688 -7.077056884765625 -7.0770416259765625
6.063281536102295 -2.294200897216797 -4.398778915405273 -1.9475860595703125 -5.490623474121094 -5.490509033203125
6.013682842254639 -2.140162229537964 -4.286874294281006 -1.740002155303955 -7.708763122558594 -7.708824157714844
6.056893825531006 -2.086667060852051 -4.329707145690918 -1.5630302429199219 -3.152256

6.01199197769165 -2.1882803440093994 -4.299367904663086 -1.2047653198242188 -1.5101699829101562 -1.510162353515625
6.024225234985352 -2.256056070327759 -4.489656925201416 -2.1157302856445312 -3.070718765258789 -3.070772171020508
6.021371364593506 -2.2262611389160156 -4.407971382141113 -3.0254364013671875 -8.33857536315918 -8.33863639831543
6.000516414642334 -2.226447105407715 -4.476352691650391 -0.5669479370117188 -2.3306045532226562 -2.3306198120117188
6.017801284790039 -2.2522053718566895 -4.492978572845459 -3.3592376708984375 -1.5117874145507812 -1.5117568969726562
6.020867824554443 -2.1966488361358643 -4.249969959259033 -3.0810317993164062 -9.055950164794922 -9.055904388427734
6.05754280090332 -2.0860719680786133 -4.123812198638916 -3.1264190673828125 -1.2356185913085938 -1.2356491088867188
6.037773132324219 -2.0908102989196777 -4.120652675628662 -1.0196075439453125 -1.4515838623046875 -1.451568603515625
6.026955604553223 -2.1282107830047607 -4.2291693687438965 -3.0281600952148438 

KeyboardInterrupt: 

In [3]:
train(300000, 50, 400, 8, 1)

6.023568153381348 -1.5398733615875244 -3.0662338733673096 -3.3729248046875 -8.119998931884766 -8.120044708251953
6.246954917907715 -1.6592010259628296 -3.318229913711548 -1.67840576171875 -5.932342529296875 -5.93231201171875
6.26201868057251 -1.7248412370681763 -3.4327733516693115 -1.039581298828125 -2.993560791015625 -2.99365234375
6.090731143951416 -1.5266666412353516 -3.018648147583008 -2.194061279296875 -0.1717529296875 -0.171722412109375
6.033583641052246 -1.5316256284713745 -2.9804515838623047 -2.6064891815185547 -6.3674468994140625 -6.36749267578125
6.13478946685791 -1.4749435186386108 -2.916316509246826 -3.0022125244140625 -6.666419982910156 -6.666313171386719
6.113887786865234 -1.424017310142517 -2.8723695278167725 -2.0409088134765625 -3.239349365234375 -3.2393646240234375
6.048005104064941 -1.4640358686447144 -2.87858247756958 -1.56134033203125 -4.7127685546875 -4.7126922607421875
6.094243049621582 -1.5643430948257446 -3.0663156509399414 -2.7397003173828125 -10.78144073486328

6.018649578094482 -1.3719419240951538 -2.762350082397461 -1.1684722900390625 -4.16229248046875 -4.162200927734375
6.039662837982178 -1.3749397993087769 -2.7595462799072266 -1.517181396484375 -1.673252820968628 -1.673283338546753
6.005737781524658 -1.433632731437683 -2.818175792694092 -0.769134521484375 -2.5508193969726562 -2.5508346557617188
6.007047176361084 -1.429152488708496 -2.8886239528656006 -2.5822081565856934 -0.8279571533203125 -0.8279266357421875
5.993679523468018 -1.4437328577041626 -2.887322425842285 -0.8335113525390625 -1.08734130859375 -1.087310791015625
6.008981704711914 -1.5174288749694824 -2.9854400157928467 -1.247772216796875 -2.1790084838867188 -2.1789627075195312
6.009130954742432 -1.4192395210266113 -2.836617946624756 -1.703765869140625 -0.66143798828125 -0.661468505859375
5.977314472198486 -1.4646764993667603 -2.934077024459839 -1.5086212158203125 -3.477081298828125 -3.4770965576171875
6.013271331787109 -1.4510958194732666 -2.874204158782959 -1.4098663330078125 -3

6.064366340637207 -1.6675950288772583 -3.3806064128875732 -1.4951839447021484 -8.86669921875 -8.866455078125
6.049645900726318 -1.613226294517517 -3.183746576309204 -1.9566650390625 -2.272186279296875 -2.272186279296875
5.985975742340088 -1.6316454410552979 -3.208092212677002 -2.22015380859375 -2.2812652587890625 -2.28125
6.027978420257568 -1.5017495155334473 -3.0302364826202393 -1.5544800758361816 -1.9788970947265625 -1.9788970947265625
6.031459808349609 -1.634117841720581 -3.185354232788086 -1.1089324951171875 -1.9592742919921875 -1.9592742919921875
6.002566814422607 -1.5618538856506348 -3.1235218048095703 -1.3429718017578125 -2.1529541015625 -2.152923583984375
6.046148777008057 -1.670635461807251 -3.291520595550537 -2.4424896240234375 -1.7820587158203125 -1.7819976806640625
6.012083053588867 -1.6392203569412231 -3.2190709114074707 -1.5404510498046875 -3.28326416015625 -3.2832794189453125
6.021412372589111 -1.6796472072601318 -3.3442578315734863 -2.1482391357421875 -7.368461608886719

6.061927318572998 -1.5565569400787354 -3.1260881423950195 -2.3364410400390625 -0.6572265625 -0.657257080078125
6.053489685058594 -1.5435351133346558 -3.076037883758545 -0.978302001953125 -0.3608245849609375 -0.3607940673828125
6.0218424797058105 -1.582396149635315 -3.1365437507629395 -2.128936767578125 -0.6977691650390625 -0.69775390625
5.994687557220459 -1.633139729499817 -3.26900315284729 -0.4818267822265625 -10.26348876953125 -10.263397216796875
6.021116733551025 -1.7079992294311523 -3.427185297012329 -1.5437207221984863 -0.6051979064941406 -0.6051673889160156
6.000977516174316 -1.7082159519195557 -3.3897509574890137 -1.3397979736328125 -0.8755912780761719 -0.8755302429199219
6.018209934234619 -1.5855412483215332 -3.194596529006958 -2.2794036865234375 -6.4210662841796875 -6.4210052490234375
5.994576930999756 -1.4497209787368774 -2.8257241249084473 -2.1460342407226562 -2.7043304443359375 -2.70440673828125
6.005459308624268 -1.519565224647522 -2.9421792030334473 -1.970733642578125 -3.

6.01433801651001 -1.642077088356018 -3.298593282699585 -2.2126007080078125 -3.0997543334960938 -3.0997314453125
6.045698165893555 -1.5788015127182007 -3.1298890113830566 -0.2984466552734375 -1.826873779296875 -1.826904296875
6.032881736755371 -1.5988575220108032 -3.2123446464538574 -1.216400146484375 -2.16357421875 -2.16357421875
6.040327548980713 -1.6191649436950684 -3.183516263961792 -1.53558349609375 -0.28643035888671875 -0.28646087646484375
6.036682605743408 -1.5393481254577637 -3.0122523307800293 -2.1407928466796875 -3.137603759765625 -3.137664794921875
6.022494792938232 -1.4920953512191772 -2.946065902709961 -2.2370071411132812 -0.9682083129882812 -0.9681777954101562
5.986566543579102 -1.4572728872299194 -2.8822789192199707 -2.0501441955566406 -3.0030364990234375 -3.0030364990234375
6.047396183013916 -1.4682682752609253 -2.8310868740081787 -1.735992431640625 -1.5905303955078125 -1.590606689453125
6.003820896148682 -1.4855245351791382 -2.9506165981292725 -2.185211181640625 -4.6184

...

5.952221393585205 -0.8138006329536438 -1.644484519958496 -1.19195556640625 -0.729339599609375 -0.729339599609375
5.976369380950928 -0.8222996592521667 -1.6523743867874146 -0.096282958984375 -1.2835693359375 -1.2835693359375
5.979599475860596 -0.8749046325683594 -1.7287285327911377 -1.3930206298828125 -2.274688720703125 -2.274871826171875
5.987456321716309 -0.8358843922615051 -1.651026725769043 -0.64959716796875 -0.53765869140625 -0.5377197265625
5.985710144042969 -0.844125509262085 -1.668681263923645 -1.5965118408203125 -1.315185546875 -1.3153076171875
5.960379600524902 -0.8724953532218933 -1.7187825441360474 -0.20489501953125 -0.724090576171875 -0.724029541015625
5.959041118621826 -0.9011206030845642 -1.8006325960159302 -0.80889892578125 -0.357757568359375 -0.357757568359375
5.969557762145996 -0.963100016117096 -1.9216346740722656 -0.817840576171875 -1.99798583984375 -1.998016357421875
5.97389030456543 -0.940506100654602 -1.869964599609375 -0.50054931640625 -0.30255126953125 -0.302551

5.930168151855469 -0.7802513241767883 -1.5692058801651 -0.66802978515625 -1.2147216796875 -1.21484375
5.956928730010986 -0.723626971244812 -1.4277361631393433 -0.4873809814453125 -0.4898681640625 -0.489990234375
5.969399929046631 -0.7626098990440369 -1.5212599039077759 -0.2657470703125 -0.0345916748046875 -0.0345916748046875
5.962835788726807 -0.828805148601532 -1.6342389583587646 -0.779541015625 -0.4066162109375 -0.4066162109375
5.945806980133057 -0.7234289646148682 -1.5104122161865234 -0.296173095703125 -0.7987060546875 -0.79864501953125
5.960546970367432 -0.7475605010986328 -1.4966955184936523 -0.3665924072265625 -0.5748291015625 -0.57476806640625
5.936761379241943 -0.7145489454269409 -1.4225572347640991 -0.80340576171875 -0.032257080078125 -0.032257080078125
5.947836875915527 -0.7180830240249634 -1.4373310804367065 -0.7660388946533203 -0.4602813720703125 -0.4601287841796875
5.932556629180908 -0.6649529933929443 -1.3714770078659058 -0.03179931640625 -1.1134796142578125 -1.1134185791

5.995999813079834 -0.8433498740196228 -1.7589197158813477 -0.204559326171875 -4.73065185546875 -4.73065185546875
5.993971824645996 -0.9275479912757874 -1.7824584245681763 -0.3496856689453125 -2.9322509765625 -2.9326171875
5.995725154876709 -0.8808916807174683 -1.7301191091537476 -0.20220947265625 -1.63226318359375 -1.632293701171875
5.998725414276123 -0.8273365497589111 -1.5908757448196411 -2.5444107055664062 -3.3861083984375 -3.38604736328125
5.991199016571045 -0.8117623925209045 -1.6103202104568481 -0.75335693359375 -4.573871612548828 -4.574043273925781
5.947190284729004 -0.757511556148529 -1.536368727684021 -0.739654541015625 -0.120361328125 -0.12030029296875
5.949697017669678 -0.8294317722320557 -1.6173791885375977 -0.7796630859375 -0.790374755859375 -0.790313720703125
5.975479602813721 -0.7737070918083191 -1.5813312530517578 -0.4842529296875 -1.6661376953125 -1.66632080078125
5.975475788116455 -0.843211829662323 -1.7208304405212402 -2.083038330078125 -0.54327392578125 -0.543334960

5.959279537200928 -0.7367222309112549 -1.4632844924926758 -0.34429931640625 -3.7830810546875 -3.7830810546875
5.965009689331055 -0.7789323329925537 -1.5684378147125244 -1.013946533203125 -0.35546875 -0.35552978515625
5.956852436065674 -0.7608912587165833 -1.5246585607528687 -0.947509765625 -2.59552001953125 -2.5955810546875
5.9723944664001465 -0.83710777759552 -1.6875030994415283 -0.1151123046875 -0.0799560546875 -0.0799560546875
5.966694355010986 -0.8450783491134644 -1.7156082391738892 -0.730377197265625 -0.3553466796875 -0.3553466796875
5.99517822265625 -0.8650322556495667 -1.7244269847869873 -0.990509033203125 -1.513916015625 -1.513916015625
5.975976943969727 -0.8671786189079285 -1.7202308177947998 -0.5518798828125 -0.05560302734375 -0.05560302734375
5.97871208190918 -0.8356214761734009 -1.674102783203125 -0.045562744140625 -4.3629150390625 -4.3626708984375
5.950334548950195 -0.7639632225036621 -1.5576890707015991 -1.4454345703125 -0.61151123046875 -0.611572265625
5.956079006195068 

KeyboardInterrupt: 

In [2]:
NN.load_state_dict(torch.load('TSP_50points_big_6.00'))

<All keys matched successfully>

In [5]:
def evaluate(epochs = 30000, npoints = 10, batchsize = 100, nsamples = 8, negative_cutoff = 1):
    NN.eval()
    for i in range(epochs):
        env.reset(npoints, batchsize, nsamples)
        """include the boundary points, kinda makes sense that they should contribute (atm only in the encoder, difficult to see how in the decoder)"""
        memory = NN.encode(env.points) #[npoints, batchsize * nsamples, emsize]
        #### #### #### remember to include tgt.detach() when reinstate with torch.no_grad()
        tgt = NN.start_token.unsqueeze(0).unsqueeze(1).expand(1, batchsize * nsamples, -1).detach() #[1, batchsize * nsamples, emsize]
        
        NN.eval()
        #with torch.no_grad(): #to speed up computation, selecting routes is done without gradient
        with torch.no_grad():
            for j in range(0, npoints):
                #### #### #### remember to include memory.detach() when reinstate with torch.no_grad()
                _, logits = NN.decode_next(memory.detach(), tgt, env.points_mask)
                next_point = env.sampleandgreedy_point(logits)
                """
                for inputing the previous embedding into decoder
                """
                tgt = torch.cat([tgt, memory.gather(0, next_point.unsqueeze(0).unsqueeze(2).expand(1, -1, memory.size(2)))]) #[nsofar, batchsize * nsamples, emsize]
                """
                for inputing the previous decoder output into the decoder (allows for an evolving strategy, but doesn't allow for fast training
                """
                ############

        env.laststep()
        
        _, logprob = NN.calculate_logprob(memory, env.points_sequence) #[batchsize * nsamples]
        
        """
        clip logprob so doesn't reinforce things it already knows
        TBH WANT SOMETHING DIFFERENT ... want to massively increase training if find something unexpected and otherwise not
        """
        greedy_prob = logprob.view(batchsize, nsamples)[:, -1].detach() #[batchsize]
        greedy_baseline = env.cost.view(batchsize, nsamples)[:, -1] #[batchsize], greedy sample
        
        print(greedy_baseline.mean().item(), logprob.view(batchsize, nsamples)[:, -1].mean().item(), logprob.view(batchsize, nsamples)[:, :-1].mean().item(), logprob[batchsize - 1].item(), logprob[0].item(), env.logprob[0].item())
     

In [ ]:
evaluate(10, 1000, 1, 2)

36.72380447387695 -184.73703002929688 -335.1563415527344 -335.1563415527344 -335.1563415527344 -335.1607666015625
35.7786979675293 -206.96295166015625 -318.55487060546875 -318.55487060546875 -318.55487060546875 -318.55682373046875
35.84605407714844 -159.57765197753906 -294.0115661621094 -294.0115661621094 -294.0115661621094 -294.0093078613281
37.04632568359375 -160.7367706298828 -323.0457763671875 -323.0457763671875 -323.0457763671875 -323.0462341308594
35.231807708740234 -172.41758728027344 -368.943603515625 -368.943603515625 -368.943603515625 -368.9453125
36.44965362548828 -180.2964324951172 -347.7308349609375 -347.7308349609375 -347.7308349609375 -347.7281799316406
36.8441162109375 -176.00906372070312 -301.78753662109375 -301.78753662109375 -301.78753662109375 -301.7870178222656
35.09375 -193.97610473632812 -356.5091552734375 -356.5091552734375 -356.5091552734375 -356.5086669921875
36.04238510131836 -167.0640106201172 -392.3709411621094 -392.3709411621094 -392.3709411621094 -392.369